In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from utils import *
import mlflow
pd.options.plotting.backend = "plotly"

# Data

In [2]:
df_1 = pd.read_csv("/Users/tsfelg/repos/mlops_zoomcamp_project/2020_PT_Region_Mobility_Report.csv")
df_2 = pd.read_csv("/Users/tsfelg/repos/mlops_zoomcamp_project/2021_PT_Region_Mobility_Report.csv")
df_3 = pd.read_csv("/Users/tsfelg/repos/mlops_zoomcamp_project/2022_PT_Region_Mobility_Report.csv")
df = pd.concat([df_1,df_2,df_3])

In [3]:
df = clean_data(df)
df = feature_extraction(df)
df = df.dropna()

In [4]:
df_train = df.loc["2020":"2022-07"]
df_test = df.loc["2022-08"]

X_train = df_train.drop(columns=["y"])
X_test = df_test.drop(columns=["y"])

y_train = df_train.y
y_test = df_test.y

# Training

In [5]:
model = LinearRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
df.loc[X_test.index, "y_pred"] = y_pred

# Export

In [6]:
%env MLFLOW_TRACKING_URI=https://mlflow-gcp-2-r5wfb53nxq-uc.a.run.app/
%env MLFLOW_TRACKING_USERNAME=mlflow
%env MLFLOW_TRACKING_PASSWORD=password123
%env MLFLOW_EXPERIMENT_NAME=my_first_experiment
%env GOOGLE_APPLICATION_CREDENTIALS=/Users/tsfelg/repos/mlflow-for-gcp/secrets/credentials.json

env: MLFLOW_TRACKING_URI=https://mlflow-gcp-2-r5wfb53nxq-uc.a.run.app/
env: MLFLOW_TRACKING_USERNAME=mlflow
env: MLFLOW_TRACKING_PASSWORD=password123
env: MLFLOW_EXPERIMENT_NAME=my_first_experiment
env: GOOGLE_APPLICATION_CREDENTIALS=/Users/tsfelg/repos/mlflow-for-gcp/secrets/credentials.json


In [7]:
mlflow.set_experiment('retail_forecasting')
mlflow.sklearn.log_model(model, artifact_path="model")

/opt/homebrew/Caskroom/miniforge/base/envs/mlops_zoomcamp_project/lib/python3.9/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


ModelInfo(artifact_path='model', flavors={'python_function': {'model_path': 'model.pkl', 'loader_module': 'mlflow.sklearn', 'python_version': '3.9.13', 'env': 'conda.yaml'}, 'sklearn': {'pickled_model': 'model.pkl', 'sklearn_version': '1.1.2', 'serialization_format': 'cloudpickle', 'code': None}}, model_uri='runs:/52bf59cb151d46ac8827defb5ffa8363/model', model_uuid='ce7d8e9a5b3f4768971dd240f37f13dd', run_id='52bf59cb151d46ac8827defb5ffa8363', saved_input_example_info=None, signature_dict=None, utc_time_created='2022-08-31 11:21:34.662961', mlflow_version='1.28.0')

In [8]:
mlflow.end_run()